In [3]:
!pip install nltk mlflow

  Using cached mlflow-3.8.1-py3-none-any.whl.metadata (31 kB)
  Using cached mlflow_skinny-3.8.1-py3-none-any.whl.metadata (31 kB)
  Using cached mlflow_tracing-3.8.1-py3-none-any.whl.metadata (19 kB)
  Using cached flask_cors-6.0.2-py3-none-any.whl.metadata (5.3 kB)
  Using cached flask-3.1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached alembic-1.18.1-py3-none-any.whl.metadata (7.2 kB)
  Using cached cryptography-46.0.3-cp311-abi3-win_amd64.whl.metadata (5.7 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached huey-2.6.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached matplotlib-3.10.8-cp313-cp313-win_amd64.whl.metadata (52 kB)
  Using cached numpy-2.4.1-cp313-cp313-win_amd64.whl.metadata (6.6 kB)
  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached pyarrow-22.0.0-cp313-cp313-win_amd64.whl.metadata (3.3 kB)
  Using cached scikit_learn-1.8.0-cp313-cp

In [10]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [18]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
21,What on earth has become of our dear Ramu? Is ...,negative
948,"Wow! An amazing, lost piece of Australiana AND...",positive
983,There is no way to put into words just how bad...,negative
358,Where oh where to begin in describing the comp...,negative
477,THHE2 is entertaining in that you'll laugh a l...,negative


In [19]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [20]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SAISHN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SAISHN\AppData\Roaming\nltk_data...


True

In [21]:
df = normalize_text(df)
df.head()

,review,sentiment
21,earth become dear ramu man made sarkar satya c...,negative
948,wow amazing lost piece australiana lost s glam...,positive
983,way put word bad shapeless paceless laughless ...,negative
358,oh begin describing comprehensive wretchedness...,negative
477,thhe entertaining laugh lot cringe probably sa...,negative


In [22]:
df['sentiment'].value_counts()

sentiment
negative    253
positive    247
Name: count, dtype: int64

In [23]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [24]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
21,earth become dear ramu man made sarkar satya c...,0
948,wow amazing lost piece australiana lost s glam...,1
983,way put word bad shapeless paceless laughless ...,0
358,oh begin describing comprehensive wretchedness...,0
477,thhe entertaining laugh lot cringe probably sa...,0


In [25]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [34]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
!pip install dagshub

  Using cached dagshub-0.6.4-py3-none-any.whl.metadata (12 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached rich-14.2.0-py3-none-any.whl.metadata (18 kB)
  Using cached dacite-1.6.0-py3-none-any.whl.metadata (14 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached gql-4.0.0-py3-none-any.whl.metadata (10 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached treelib-1.8.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pathvalidate-3.3.1-py3-none-any.whl.metadata (12 kB)
  Using cached boto3-1.42.30-py3-none-any.whl.metadata (6.8 kB)
  Using cached semver-3.0.4-py3-none-any.whl.metadata (6.8 kB)
  Using cached dagshub_annotation_converter-0.1.15-py3-none-any.whl.metadata (3.2 kB)
  Using cached lxml-6.0.2-cp313-cp313-win_amd64.whl.metadata (3.7 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using 

In [36]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/nsaishiva18/mlops_imdb.mlflow')
dagshub.init(repo_owner='nsaishiva18', repo_name='mlops_imdb', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2026-01-19 19:24:39,738 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/nsaishiva18/mlops_imdb "HTTP/1.1 200 OK"


Initialized MLflow to track repo "nsaishiva18/mlops_imdb"

2026-01-19 19:24:39,744 - INFO - Initialized MLflow to track repo "nsaishiva18/mlops_imdb"


Repository nsaishiva18/mlops_imdb initialized!

2026-01-19 19:24:39,746 - INFO - Repository nsaishiva18/mlops_imdb initialized!


<Experiment: artifact_location='mlflow-artifacts:/33cda0c71c024cbeb3c6f647968dad34', creation_time=1768830328121, experiment_id='0', last_update_time=1768830328121, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [37]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2026-01-19 19:25:08,258 - INFO - Starting MLflow run...
2026-01-19 19:25:08,653 - INFO - Logging preprocessing parameters...
2026-01-19 19:25:09,760 - INFO - Initializing Logistic Regression model...
2026-01-19 19:25:09,761 - INFO - Fitting the model...
2026-01-19 19:25:09,787 - INFO - Model training complete.
2026-01-19 19:25:09,788 - INFO - Logging model parameters...
2026-01-19 19:25:10,136 - INFO - Making predictions...
2026-01-19 19:25:10,138 - INFO - Calculating evaluation metrics...
2026-01-19 19:25:10,160 - INFO - Logging evaluation metrics...
2026-01-19 19:25:11,650 - INFO - Saving and logging the model...
2026/01/19 19:25:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026-01-19 19:25:26,986 - INFO - Model training and logging completed in 18.33 seconds.
2026-01-19 19:25:26,987 - INFO - Accuracy: 0.62
2026-01-19 19:25:26,988 - INFO - Precision: 0.6590909090909091
2026-01-19 19:25:26,988 - INFO - Recall: 0.5576923076923077
2026-01-19 

🏃 View run ambitious-colt-326 at: https://dagshub.com/nsaishiva18/mlops_imdb.mlflow/#/experiments/0/runs/8b6196c489c14e4cb9e1f4aec4355440
🧪 View experiment at: https://dagshub.com/nsaishiva18/mlops_imdb.mlflow/#/experiments/0
